In [1]:
import time
import MGF2PyTorch
from torch.utils.data import DataLoader

## [1] Parsing MGF files
I implemented a class called `MGF2PyTorch` that supports multiple methods for parsing, storing, and loading ms/ms spectra from MGF files and convert them into a format suitable for PyTorch.

In [2]:
converter = MGF2PyTorch.MGF2PyTorch()
mgf_file = "b1938_293T_proteinID_02B_QE3_122212.mgf"
hdf5_file = "output_flat.hdf5"

The first parser is a manual parser that reads the MGF file line by line and extracts the relevant information. This parser is fast, making it ideal for large files. It is also flexible, allowing for easy modifications to the parsing logic if needed.

In [3]:
start_time = time.time()
spectra_manual = converter.parse_mgf(mgf_file)
end_time = time.time()
print(f"Manual parsing took {end_time - start_time:.2f} seconds")

Manual parsing took 12.76 seconds


The second parser uses the `pyteomics` library to read the MGF file. This appriach provides a standardized way to read MGF files and is useful for compatibility with other tools.

In [4]:
start_time = time.time()
spectra_pyteomics = converter.read_mgf_pyteomics(mgf_file)
end_time = time.time()
print(f"Pyteomics parsing took {end_time - start_time:.2f} seconds")

Pyteomics parsing took 14.99 seconds


The third parsing function combines manual parser with storing to an HDF5 file using a flat representation. The following fields are flattened and stored: m/z values, Intensities, Peak list lengths, Charges, precursor m/z values, and retention times.

In [5]:
start_time = time.time()
converter.parse_and_store_flat_hdf5(mgf_file, hdf5_file)
end_time = time.time()
print(f"Storing to HDF5 took {end_time - start_time:.2f} seconds")

Storing to HDF5 took 12.88 seconds


## [2] Querying the spectra
In the MGF2PyTorch class, I implemented two example functions of `filter_spectra_by_charge` and `filter_peaks_by_mz` to demonstrate meta-data filtering and peak filtering. These functions are useful for quickly extracting relevant information before model training.

In [10]:
# Filter spectra to select only those with a charge of 2.
high_charge_spectra = MGF2PyTorch.filter_spectra_by_charge(spectra_manual, charge_value=2)
filtered_spectra = MGF2PyTorch.filter_peaks_by_mz(spectra_manual[0], 100, 200)
print("Filtered Peaks:\n", filtered_spectra)
print("Spectrum title:\n", spectra_manual[0]['title'])

Filtered Peaks:
 [[  112.08773   4129.667   ]
 [  113.07166   1194.3569  ]
 [  115.08704  52450.684   ]
 [  115.094635  2902.2378  ]
 [  116.01717   2280.9895  ]
 [  116.091225  1693.554   ]
 [  129.10263  49685.86    ]
 [  129.11165   3384.201   ]
 [  130.0868   10056.626   ]
 [  133.04329  15631.203   ]
 [  137.07132   1505.9761  ]
 [  139.08687   1986.8506  ]
 [  147.1128   21817.438   ]
 [  148.34206   1413.1466  ]
 [  149.5296    1493.5461  ]
 [  158.09259   1871.8987  ]
 [  161.03833   1878.8705  ]
 [  165.066     2564.3567  ]
 [  167.08179   3148.0422  ]
 [  171.05885   4739.223   ]
 [  175.1193   14015.725   ]
 [  178.0644    4805.7373  ]
 [  185.07455   3086.0195  ]
 [  186.12479   1802.6827  ]
 [  188.03183   1586.6492  ]]
Spectrum title:
 b1938_293T_proteinID_02B_QE3_122212.1701.1701.2


## [3] PyTorch integration
I implemented a custom SpectraDataset class to wrap the HDF5 data for use in PyTorch pipelines. This class includes context management methods (__enter__ and __exit__) to ensure that HDF5 file handles are properly managed and closed, preventing resource leaks during extended training sessions. Furthermore, a custom collate function is used in the DataLoader to pad variable-length sequences, thereby ensuring uniform batch processing for training neural network models. This makes the dataset compatible with PyTorch's DataLoader for efficient and scalable data loading.

In [6]:
with MGF2PyTorch.SpectraDataset(hdf5_file) as dataset:
    print(f"Number of spectra in HDF5: {len(dataset)}")
    dataloader = DataLoader(dataset, batch_size=4, collate_fn=MGF2PyTorch.collate_fn, shuffle=False)

    for batch in dataloader:
        print("Batch keys:", batch.keys())
        print("Batch m/z shape:", batch['mz'].shape)
        print("Batch intensity shape:", batch['intensity'].shape)
        print("Batch charge:", batch['charge'])
        print("Batch precursor:", batch['precursor'])
        print("Batch retention time:", batch['rt'])
        break

Number of spectra in HDF5: 45642
Batch keys: dict_keys(['mz', 'intensity', 'charge', 'precursor', 'rt'])
Batch m/z shape: torch.Size([4, 422])
Batch intensity shape: torch.Size([4, 422])
Batch charge: tensor([2, 2, 2, 4])
Batch precursor: tensor([360.6677, 317.1633, 322.1191, 356.4366])
Batch retention time: tensor([1499.2566, 1501.8916, 1503.2412, 1668.9448])


# Performance
## Parsing efficiency
The manual parser is approximately 15% faster than the pyteomics parser which is the reason I implemented it within the third parser which combines it with flatting and directly stores data to HDF5 file. This avoids redundant parsing steps and keeps memory usage low.
## I/O efficiency
The HDF5 file format is optimized for fast I/O operations, making it suitable for large datasets. I chose this method as it is efficient especially in machine learning applications. Another method was combination of Numpy arrays and Parquet but based on the use case I chose this approach as it better supports variable-length spectra in a single file.
## Scalability and resource management
The integration of context management within the SpectraDataset class ensures that resources (e.g., file handles) are correctly managed even in long-running sessions or when processing multiple files in parallel. I chose it to minimal I/O bottlenecks and to ensure that the system can handle larger datasets without running into memory issues. This is particularly important in machine learning applications where large datasets are common.